In [1]:
import utils
import emission
import transition
import viterbi

import copy
from random import randint, choice
import numpy as np

In [2]:
LOWER = True
NORM_TENSE = True
REP_NUM = True
REP_YEAR = False
REP_SYM = True

In [3]:
dataset_folder = "data/EN/"
train_data = dataset_folder + "train"
lines = utils.read_file_to_lines(train_data)

eval_data = dataset_folder + "dev.out"
lines_2 = utils.read_file_to_lines(eval_data)

lines += lines_2

In [4]:
emission_data = emission.generate_emission_table(lines,
                                                 lower=LOWER,
                                                 norm_tense=NORM_TENSE,
                                                 replace_number=REP_NUM,
                                                 replace_year=REP_YEAR,
                                                 replace_symbol=REP_SYM)
hashmap = emission_data["x_hashmap"]
word_freq = emission_data["x_word_freq"]
smoothed_hashmap = utils.add_unk(hashmap, word_freq, k=2)
emission_data["x_hashmap"] = smoothed_hashmap

x_vocab = utils.get_emission_vocab(smoothed_hashmap)
print("Vocab size:", len(x_vocab))

Vocab size: 7443


In [5]:
transition_pairs = transition.generate_transition_pairs(lines)

In [6]:
y_pairs = transition_pairs["Y_pairs"]
y_vocab = transition_pairs["y_vocab"]
y_freq = transition_pairs["y_freq"]

In [7]:
transition_data = transition.generate_transition_data(y_pairs, y_vocab)

In [8]:
hmm = viterbi.HMM()
hmm.fit_word_tokenizer(x_vocab)
hmm.fit_pos_tokenizer(y_vocab)
hmm.build_transition_weights(y_freq, transition_data)
hmm.build_emission_weights(emission_data)

In [9]:
train_data = "data/TEST/EN/test.in"
lines = utils.read_file_to_lines(train_data)

sentences = []

while len(lines) > 1:
    sentence_break = lines.index("")
    sentence_xy = lines[:sentence_break]
    words = [utils.preprocess_text(token,
                                   lower=LOWER,
                                   norm_tense=NORM_TENSE,
                                   replace_number=REP_NUM,
                                   replace_year=REP_YEAR,
                                   replace_symbol=REP_SYM)
             for token in sentence_xy]
    sentence = " ".join(words).strip()
    sentences.append(sentence)
    lines = lines[sentence_break+1:]

In [10]:
new_words = []

for line in sentences:
    for word in line.split(" "):
        if word not in x_vocab:
            new_words.append(word)
        
new_words = list(set(new_words))
new_words.sort()
print("New words", len(new_words))

New words 2165


In [11]:
# only for the progress bar!
try:
    from tqdm import tqdm
    USE_TQDM = True
except Exception as e:
    print(e, "TQDM import error, disable progress bar")

if USE_TQDM:
    sentences_it = tqdm(sentences)
else:
    sentences_it = sentences

  0%|          | 0/2189 [00:00<?, ?it/s]

In [12]:
preds = []

for line in sentences_it:
    pred = hmm.viterbi_predict(line)
    pred = hmm.pos_tokens_to_labels(pred)
    preds.append(pred)
    
assert len(sentences) == len(preds)

100%|██████████| 2189/2189 [00:47<00:00, 46.41it/s]


In [13]:
train_data = "data/TEST/EN/test.in"
lines = utils.read_file_to_lines(train_data)

og_sentences = []

while len(lines) > 1:
    sentence_break = lines.index("")
    sentence_xy = lines[:sentence_break]
    words = [token.strip()
             for token in sentence_xy]
    sentence = " ".join(words).strip()
    og_sentences.append(sentence)
    lines = lines[sentence_break+1:]

In [14]:
outfile = "data/TEST/EN/test.out"

with open(outfile, "w") as f:
    for sentence, pred in zip(og_sentences, preds):
        word_array = sentence.split(" ")
        try:
            assert len(word_array) == len(pred)
            for i, word in enumerate(word_array):
                f.write(word + " " + pred[i] +"\n")
        except:
            print(word_array)
            print(pred)
            break
        f.write("\n")